In [1]:

import sys
sys.path.append("D:/Projekte/Sim/src")
import sim
import timeit

## Ablagen

In [2]:
agv = sim.Jig("AGV-Bhf HC")
sim.BufferSource("HC-Einlauf", 60, 8, agv, getNextPart=sim.src_rework_10)

hc_rot1 = sim.RotBuffer("HC Drehspeicher 1", 3, 6, 5)
hc_rot2 = sim.RotBuffer("HC Drehspeicher 2", 3, 6, 5)

hc_drn = sim.Jig("HC Drain")
drn = sim.BufferDrain("Hc Drain", 72, parent=hc_drn)
drn.waitingfor=500

hand_special = sim.Jig("Handling Special")
hand_mess = sim.Jig("Handling Mess")

imt = sim.Jig("IMT")
sim.BufferWorkOn(
    "Messen",
    150,
    parent=imt,
    condition=sim.cond_inline,
    modifier=sim.mod_inline_measure,
)

na1 = sim.Jig("Nacharbeit 1")
sim.BufferWorkOn(
    "Nacharbeit",
    69,
    condition=sim.cond_rework,
    modifier=sim.mod_rework_done,
    parent=na1,
)

na2 = sim.Jig("Nacharbeit 2")
sim.BufferWorkOn(
    "Nacharbeit",
    69,
    condition=sim.cond_rework,
    modifier=sim.mod_rework_done,
    parent=na2,
)

aus_hc = sim.Buffer("Ausschleuse 1", 3)
sim.BufferWorkOn(
    "Ultraschall", 3000, condition=sim.cond_us, modifier=sim.mod_us_testing
)


## Roboter + Programme

In [3]:

verteiler = sim.Rob("Verteiler HC")
sim.RobHandlingBetter(
    "Verteilen",
    [
        sim.Pick(agv, 10),
        [
            sim.Place(hand_mess, time = 5, timeto = 15, add_condition=sim.cond_inline),
            sim.Place(hand_special, time = 5, timeto = 15, add_condition=lambda x, **kwargs: sim.cond_rework(x) or sim.cond_us(x)),
            sim.Place(hc_rot1, time = 5, timeto = 20, position = 1),
            sim.Place(hc_rot1, time = 5, timeto = 20, position = 5),
        ]
    ],
    parent=verteiler,
)
sim.RobHandlingConditional(
    "Aus Mess",
    hand_mess,
    [hc_rot1, hc_rot2],
    conditions=[sim.cond_all],
    pick_dur=5,
    transport_durs=[15, 15],
    place_durs=[5, 5],
    places_at_position=[1, 5],
    parent=verteiler,
)
sim.RobHandlingConditional(
    "Aus Specal",
    hand_special,
    [hc_rot1, hc_rot2],
    conditions=[sim.cond_all],
    pick_dur=5,
    transport_durs=[15, 15],
    place_durs=[5, 5],
    places_at_position=[1, 5],
    parent=verteiler,
)


abgabe_hc = sim.Rob("Abgabe HC")
sim.RobHandling(
    "Abgabe aus 1", hc_rot1, hc_drn, 5, 10, 5, parent=abgabe_hc, take_from_position=3
)
sim.RobHandling(
    "Abgabe aus 2", hc_rot1, hc_drn, 5, 10, 5, parent=abgabe_hc, take_from_position=2
)

sonder = sim.Rob("SonderRobbie")
sim.RobHandling("Mess hin", hand_mess, imt, 5, 20, 5, parent=sonder)
sim.RobHandling("Mess zurück", imt, hand_mess, 5, 20, 5, parent=sonder)
sim.RobHandling("Aus NA 1", na1, hand_special, 5, 15, 5, parent=sonder)
sim.RobHandling("Aus NA 2", na2, hand_special, 5, 15, 5, parent=sonder)

sim.RobHandling("Aus Ausschleuse", aus_hc, hand_special, 5, 20, 5, parent=sonder)
sim.RobHandlingConditional(
    "Aus Sonderablage",
    hand_special,
    [aus_hc, na1, na2],
    conditions=[sim.cond_us, sim.cond_all],
    pick_dur=5,
    transport_durs=[20, 15, 15],
    place_durs=[5, 5, 5],
    parent=sonder,
)


## Simulation

In [4]:
start = timeit.timeit()
sim.core.run(0, 5 * 60 * 60)
end = timeit.timeit()
print(end - start)


-0.0008954000000009898


## Plotting

In [5]:
sim.core.getAllTraces()

['Rotation_AGV-Bhf HC',
 'AGV-Bhf HC_Fill',
 'Rotation_HC Drehspeicher 1',
 'HC Drehspeicher 1_Fill',
 'HC Drehspeicher 1_Position',
 'Rotation_HC Drehspeicher 2',
 'HC Drehspeicher 2_Fill',
 'HC Drehspeicher 2_Position',
 'Rotation_HC Drain',
 'HC Drain_Fill',
 'Rotation_Handling Special',
 'Handling Special_Fill',
 'Rotation_Handling Mess',
 'Handling Mess_Fill',
 'Rotation_IMT',
 'IMT_Fill',
 'Rotation_Nacharbeit 1',
 'Nacharbeit 1_Fill',
 'Rotation_Nacharbeit 2',
 'Nacharbeit 2_Fill',
 'Rotation_Ausschleuse 1',
 'Ausschleuse 1_Fill',
 'Program_Verteiler HC',
 'Program_Abgabe HC',
 'Program_SonderRobbie']

In [8]:
sim.core.plotHistory(names =['HC Drain_Fill','HC Drehspeicher 2_Fill','HC Drehspeicher 1_Fill','Program_Abgabe HC', 'Rotation_HC Drehspeicher 2', 'Rotation_HC Drehspeicher 1', 'Rotation_HC Drehspeicher 2'])